<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo: Random Forest#

##Inicialización##

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

##Pre procesamiento##

In [ ]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 86 (delta 42), reused 40 (delta 15), pack-reused 0
Unpacking objects: 100% (86/86), done.
/content/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
pd.options.display.max_columns = None

df = pp.get_dataset()
df = pp.decisiontree_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)

##Entrenamiento##

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_estimators': np.arange(100, 200),
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(1, 10),
    'min_samples_leaf': np.arange(1, 3),
    'min_samples_split':np.arange(1, 3),
    'max_features': ['auto', 'sqrt', 'log2'],
}

clf = RandomForestClassifier()

rgscv = RandomizedSearchCV(
    clf, params,n_iter=200, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [ ]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8338235294117646
Best params {'n_estimators': 167, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 5, 'criterion': 'entropy'}


##Evaluación del modelo##

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, rgscv.predict(X_test)))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84        70
           1       0.86      0.63      0.73        51

    accuracy                           0.80       121
   macro avg       0.82      0.78      0.79       121
weighted avg       0.81      0.80      0.79       121



##Predicción##

In [ ]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.decisiontree_preprocessing(X_holdout)
prediction = pd.DataFrame(rgscv.predict(X),columns=['volveria'],dtype=int)
pred_formatted = pd.concat([X_holdout.id_usuario,prediction],axis=1)
pred_formatted.to_csv('randomforest.csv')
pred_formatted

,id_usuario,volveria
0,650,0
1,234,0
2,31,0
3,552,0
4,411,0
...,...,...
85,354,0
86,82,0
87,172,0
88,8,0
